# 📊 IORAG Experiment Notebook

> **Data Management Guide**: 本notebook使用标准化的路径配置。请阅读 [README_DATA.md](../../README_DATA.md) 了解数据管理最佳实践。

## 📁 目录结构
```
DarshanRAG/
├── data/                    # 数据文件（不提交git）
│   ├── raw/                 # 原始日志
│   ├── parsed/              # 解析后的日志
│   ├── archives/            # 归档文件
│   └── examples/            # 示例文件
├── knowledge_graphs/        # KG文件（不提交git）
├── experiments/
│   ├── notebooks/           # 📍 当前位置
│   ├── scripts/             # 实验脚本
│   ├── results/             # 结果输出（不提交git）
│   └── storage/             # RAG存储（不提交git）
```

## ⚠️ 重要提示
1. **不要硬编码路径** - 使用 `config_paths` 模块
2. **API Key** - 使用环境变量，不要硬编码在代码中
3. **大文件** - 所有大文件自动被git忽略，不会提交

## 🚀 Setup

In [ ]:
# ============================================================
# API Key 配置 - 使用环境变量（推荐）
# ============================================================
import os

# ⚠️ 不要硬编码API Key！使用环境变量
# 方式1: 从环境变量读取（推荐）
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    print("⚠️  OPENAI_API_KEY 环境变量未设置")
    print("请在终端中运行: export OPENAI_API_KEY='your-key-here'")
    # 或者临时设置（仅用于测试，不要提交到git）
    # os.environ["OPENAI_API_KEY"] = "your-key-here"
else:
    print("✅ OPENAI_API_KEY 已加载")
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
# ============================================================
# 路径配置 - 使用 config_paths 模块
# ============================================================
import sys
from pathlib import Path

# 添加experiments目录到Python路径
project_root = Path('/users/Minqiu/DarshanRAG')
sys.path.insert(0, str(project_root / 'experiments'))

from config_paths import PATHS, get_parsed_log_path, get_kg_path, get_storage_dir

# 配置实验数据路径
EXPERIMENT_DATE = '2025-1-1'
PARSED_LOGS = get_parsed_log_path(EXPERIMENT_DATE)
KG_FILE = get_kg_path('kg', EXPERIMENT_DATE)
STORAGE_DIR = get_storage_dir(f'lightrag_{EXPERIMENT_DATE.replace("-", "_")}')

# 确保结果输出目录存在
RESULTS_DIR = PATHS['results'] / f'iorag_{EXPERIMENT_DATE}'
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

print("✅ 路径配置完成:")
print(f"  📂 Parsed logs: {PARSED_LOGS}")
print(f"  🕸️  KG file: {KG_FILE}")
print(f"  💾 Storage: {STORAGE_DIR}")
print(f"  📊 Results: {RESULTS_DIR}")

**Parse log**

In [ ]:
! ./unpack-darshan-logs.sh /users/Minqiu/2025/1/1

In [1]:
# 假设 parent_dir 是你想要遍历的父文件夹路径
parent_dir = '/users/Minqiu/2025/1'  # 修改为你的实际父文件夹

import os
from pathlib import Path

parsed_root = Path('~/parsed-logs-1-1').expanduser()
parsed_root.mkdir(parents=True, exist_ok=True)

darshan_logs = []
for root, dirs, files in os.walk(parent_dir):
    for file in files:
        if file.endswith('.darshan') or '.darshan.' in file:
            fullpath = os.path.join(root, file)
            # 生成相对于parent_dir的路径
            rel = os.path.relpath(fullpath, parent_dir)
            output_dir = parsed_root / Path(rel).parent
            output_dir.mkdir(parents=True, exist_ok=True)
            out_txt = output_dir / (Path(file).stem + '.txt')
            darshan_logs.append((fullpath, out_txt))

from tqdm import tqdm

for in_log, out_txt in tqdm(darshan_logs, desc="Parsing darshan logs"):
    os.system(f"darshan-parser '{in_log}' > '{out_txt}'")

Parsing darshan logs:   0%|                                                                                                                                                                                         | 0/24908 [00:00<?, ?it/s]# Warning: no log utility handlers defined for module APMPI, SKIPPING.
# Warning: no log utility handlers defined for module APMPI, SKIPPING.
# Warning: no log utility handlers defined for module APMPI, SKIPPING.
# Warning: no log utility handlers defined for module APMPI, SKIPPING.
# Warning: no log utility handlers defined for module APMPI, SKIPPING.
# Warning: no log utility handlers defined for module APMPI, SKIPPING.
# Warning: no log utility handlers defined for module APMPI, SKIPPING.
# Warning: no log utility handlers defined for module APMPI, SKIPPING.
# Warning: no log utility handlers defined for module APMPI, SKIPPING.
# Warning: no log utility handlers defined for module APMPI, SKIPPING.
# Warning: no log utility handlers defined for modul

# **Darshan Log RAG**

In [4]:
%cd ~

/users/Minqiu


In [5]:
%cd pocket-rag

/users/Minqiu/pocket-rag


In [6]:
!python cookbook/07_darshan_graph_rag.py --input_path /users/Minqiu/parsed-logs-2025-1-1 --format custom_kg -o darshan_graph1_2025-1-1.json

Darshan Log GraphRAG - PocketRAG v0.4.0

Batch process folder: /users/Minqiu/parsed-logs-2025-1-1
Recursive search: True
Output format: custom_kg

Recursively searching in: /users/Minqiu/parsed-logs-2025-1-1
Found 307 log files
  Processing [1/307]: 3113659-14308890860124774052.txt... ✓ (Job 3113659)
  Processing [2/307]: 3122112-2552850857417903032.txt... ✓ (Job 3122112)
  Processing [3/307]: 3122114-9926937505719417929.txt... ✓ (Job 3122114)
  Processing [4/307]: 3122115-13561623408403974513.txt... ✓ (Job 3122115)
  Processing [5/307]: 3122115-3805848506518405580.txt... ✓ (Job 3122115)
  Processing [6/307]: 3122116-17318885988810036602.txt... ✓ (Job 3122116)
  Processing [7/307]: 3122116-18276133586591093621.txt... ✓ (Job 3122116)
  Processing [8/307]: 3122116-7550245669165394810.txt... ✓ (Job 3122116)
  Processing [9/307]: 3122116-9743626903736272634.txt... ✓ (Job 3122116)
  Processing [10/307]: 3122117-1627447760127763431.txt... ✓ (Job 3122117)
  Processing [11/307]: 3122117-279158

In [ ]:
# 索引 log 文件到向量数据库（持久化存储）
!python cookbook/index_darshan_logs.py /users/Minqiu/parsed-logs --index_path rag_storage/darshan_index

# Query KG

交互式KG查询模式

python cookbook/query_darshan_graph.py darshan_graph.json

In [9]:
from cookbook.query_darshan_graph import DarshanGraphQuery

# 加载图谱
query_tool = DarshanGraphQuery("/users/Minqiu/darshan_graph.json")

# 多跳查询
result = await query_tool.multihop_query_async(
    query="What file systems did Job 3122490 use?",
    start_entities=["Job:3122490"],
    max_hops=3
)

print(f"There are {len(result['answer_entities'])} related entities")
print(result['answer_entities'])

# 查看作业使用的文件
files = query_tool.get_job_files("3122490")
print(f"Job 3122490 used {len(files)} files")

# 查看 Rank 的 I/O 统计
stats = query_tool.get_io_stats("0")
for stat in stats[:5]:
    print(f"File: {stat['file']}")
    print(f"  reads: {stat['stats'].get('reads', 0)}")
    print(f"  bytes_read: {stat['stats'].get('bytes_read', 0)}")

# 查找使用特定文件系统的文件
files = query_tool.find_files_by_filesystem("lustre")
print(f"Files using lustre file system: {len(files)}")

# 社区检测
result = await query_tool.detect_communities_async()
communities = result['communities']
print(f"There are {len(set(communities.values()))} communities")

加载图谱: /users/Minqiu/darshan_graph.json
  ✓ 节点数: 3565
  ✓ 边数: 17719

[多跳查询]
  查询: What file systems did Job 3122490 use?
  起始实体: ['Job:3122490']
  最大跳数: 3
  ✓ 找到 4461 条路径
  ✓ 相关实体: 1454 个
There are 1454 related entities
['File:/home/1530644354', 'File:/lus/grand/3530092984', 'File:/home/2881500050', 'File:/lus/grand/1775260597', 'File:/home/2087397322', 'File:/home/1721742430', 'File:/home/4118029980', 'File:/lus/grand/3495390818', 'File:/lus/grand/2765515192', 'File:/home/345037127', 'File:/home/1665406124', 'File:/home/3148873413', 'File:/home/2559105858', 'File:/home/3782577073', 'File:/home/1680603272', 'File:/home/1771239521', 'File:/home/1445601375', 'File:/home/259281511', 'File:/lus/grand/72542267', 'File:/lus/grand/1796455576', 'File:/home/460324701', 'File:/lus/grand/2181041742', 'File:/lus/grand/3055854171', 'File:/lus/grand/3644414465', 'File:/home/18858638', 'File:/home/1228750332', 'File:/home/1148293292', 'File:/home/4289649891', 'File:/home/3039852992', 'File:/lus/grand/

/tmp/ipykernel_835368/3320579221.py:7: RuntimeWarning: coroutine 'DarshanGraphQuery.detect_communities_async' was never awaited
  result = await query_tool.multihop_query_async(


In [7]:
import json
import asyncio
from pocket_rag import MultiHopReasonerTool, CommunityDetectorTool, ToolMessage

# 加载图谱
with open('/users/Minqiu/darshan_graph.json', 'r') as f:
    graph_data = json.load(f)

graph = graph_data['graph']
nodes = graph_data['nodes']

# 多跳推理
reasoner = MultiHopReasonerTool()
result = await reasoner.process(ToolMessage(content={
    'query': 'Find all files using lustre file system',
    'start_entities': ['FileSystem:lustre'],
    'graph': graph,
    'max_hops': 3
}))

paths = result.content['paths']
answer_entities = result.content['answer_entities']

print(f"There are {len(paths)} paths")
print(f"Related entities: {answer_entities}")

# 社区检测
detector = CommunityDetectorTool()
result = await detector.process(ToolMessage(content={
    'graph': graph,
    'method': 'simple'
}))

communities = result.content['communities']
print(f"There are {len(set(communities.values()))} communities")

There are 0 paths
Related entities: []
There are 67 communities


# **Query LightRAG**

In [6]:
%cd ~

/users/Minqiu


In [7]:
%cd DarshanRAG/experiments

/users/Minqiu/DarshanRAG/experiments


In [8]:
# 构建KG
!python build_darshan_kg.py \
    --input_path /users/Minqiu/parsed-logs-2025-1-1 \
    --output_path /users/Minqiu/kg_2025-1-1.json

🔍 Searching for Darshan logs in: /users/Minqiu/parsed-logs-2025-1-1
✅ Found 307 log file(s)
📄 [1/307] Parsing: /users/Minqiu/parsed-logs-2025-1-1/3122498-16918842563362759476.txt
   ✓ Extracted 3 modules
📄 [2/307] Parsing: /users/Minqiu/parsed-logs-2025-1-1/3122501-14457461850244553889.txt
   ✓ Extracted 3 modules
📄 [3/307] Parsing: /users/Minqiu/parsed-logs-2025-1-1/3123763-16827788985070522542.txt
   ✓ Extracted 3 modules
📄 [4/307] Parsing: /users/Minqiu/parsed-logs-2025-1-1/3122115-3805848506518405580.txt
   ✓ Extracted 3 modules
📄 [5/307] Parsing: /users/Minqiu/parsed-logs-2025-1-1/3122490-15876745620081412970.txt
   ✓ Extracted 3 modules
📄 [6/307] Parsing: /users/Minqiu/parsed-logs-2025-1-1/3122504-17175317302069458101.txt
   ✓ Extracted 3 modules
📄 [7/307] Parsing: /users/Minqiu/parsed-logs-2025-1-1/3122508-6231944815274203722.txt
   ✓ Extracted 3 modules
📄 [8/307] Parsing: /users/Minqiu/parsed-logs-2025-1-1/3123603-6372769821953576231.txt
   ✓ Extracted 3 modules
📄 [9/307] Parsi

In [9]:
# 加载到LightRAG
!python load_darshan_kg.py \
    --kg_path /users/Minqiu/kg_2025-1-1.json \
    --working_dir ./lightrag_storage_2025_1_1

❌ Error: OPENAI_API_KEY environment variable not set

💡 Set your API key:
   export OPENAI_API_KEY='sk-...'


In [11]:
# 先分割KG
!python experiments/load_kg_chunked.py /users/Minqiu/darshan_graph1_2025-1.json --batch-size 50000

python: can't open file '/users/Minqiu/DarshanRAG/experiments/experiments/load_kg_chunked.py': [Errno 2] No such file or directory


In [15]:
# 然后分批加载
!python load_kg_chunked.py /users/Minqiu/darshan_graph1_2025-1.json \
    --load-batches /users/Minqiu/DarshanRAG/experiments/kg_batches

Found 7 batch files
Note: All batches will be loaded into the same LightRAG instance


Loading batch 1/7: kg_batch_001.json
Loading knowledge graph from: /users/Minqiu/DarshanRAG/experiments/kg_batches/kg_batch_001.json
Working directory: ./experiments/rag_storage
Workspace: (default)
Embedding model: sentence-transformers/all-MiniLM-L6-v2
Use local embedding: True
✅ Using local embedding model (offline, no API required)
KG contains: 50000 entities, 191672 relationships, 0 chunks

正在加载本地embedding模型: sentence-transformers/all-MiniLM-L6-v2
首次使用时会自动下载模型，请耐心等待...
tokenizer_config.json: 100%|███████████████████| 350/350 [00:00<00:00, 2.32MB/s]
vocab.txt: 232kB [00:00, 2.16MB/s]
tokenizer.json: 466kB [00:00, 6.75MB/s]
config.json: 100%|█████████████████████████████| 612/612 [00:00<00:00, 5.21MB/s]
model.safetensors: 100%|███████████████████| 90.9M/90.9M [00:02<00:00, 39.3MB/s]
✅ 模型加载完成，embedding维度: 384
Initializing LightRAG...
Note: Using local embedding model (offline, fast processing)...



*2. 启动现有的LightRAG Server*

lightrag-server --working-dir ./lightrag_storage_2025_1_1 --port 9621

*3. 浏览器打开 http://localhost:9621*

*就可以用Web UI查询和可视化图谱了*

# **Query from graphRAG**

In [6]:
!python cookbook/08_real_graph_rag_example.py --load graph_2025-1-1.json


Real GraphRAG for Darshan Logs

Using NetworkX backend (persist: darshan_graph_nx.json)
INFO:cookbook.neo4j_graph_rag:Graph loaded from darshan_graph_nx.json
INFO:cookbook.neo4j_graph_rag:NetworkX backend initialized

Loading graph from graph_2025-1-1.json...
INFO:cookbook.neo4j_graph_rag:Loaded 3565 nodes and 17719 edges
Loaded 3565 nodes and 17719 edges

Example 1: Basic Graph Operations

Graph Statistics:
  Total nodes: 3565
  Total edges: 35438
  Node types: {'Job': 67, 'Rank': 302, 'File': 3087, 'MountPoint': 26, 'FileSystem': 16, 'JobIOSummary': 67}
  Edge types: {'HAS_RANK': 822, 'USES_FILE': 20298, 'HAS_SUMMARY': 134, 'IO_POSIX': 8004, 'MOUNTED_ON': 6174, 'ON_FS': 6}

Jobs in graph: 67
  - Job:3113659: nprocs=8192
  - Job:3122112: nprocs=4
  - Job:3122114: nprocs=4
  - Job:3122115: nprocs=4
  - Job:3122116: nprocs=4
  ... and 62 more

Job Details for Job:3113659:
  Files accessed: 32
    - /lus/grand/1649248601
    - /lus/grand/1649248601
    - /lus/grand/165960639
  Ranks: 574

In [19]:
# ============================================================================
# 使用 OpenAI API 进行 GraphRAG 自然语言查询
# ============================================================================

import os
import asyncio
from openai import AsyncOpenAI

# 设置 OpenAI 客户端
client = AsyncOpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

# 定义 LLM 函数（供 GraphRAG 使用）
async def openai_llm(messages, model="gpt-5"):
    """OpenAI LLM wrapper for GraphRAG"""
    response = await client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.7
    )
    return response.choices[0].message.content

# 导入 GraphRAG 组件
from cookbook.neo4j_graph_rag import (
    create_graph_backend,
    DarshanGraphDB,
    DarshanGraphRAG,
    GraphAnalyzer
)

# 创建后端并加载图谱
backend = create_graph_backend('networkx', persist_path='darshan_graph_nx.json')
await backend.connect()

graph_db = DarshanGraphDB(backend)
await graph_db.load_from_json('graph_2025-1-1.json')

# 创建带 LLM 的 GraphRAG
rag = DarshanGraphRAG(backend, llm_func=openai_llm, model="gpt-4o-mini")

print("GraphRAG initialized!")
print(f"Graph nodes: {(await backend.get_stats())['total_nodes']}")
print(f"Graph edges: {(await backend.get_stats())['total_edges']}")


INFO:cookbook.neo4j_graph_rag:Graph loaded from darshan_graph_nx.json
INFO:cookbook.neo4j_graph_rag:NetworkX backend initialized
INFO:cookbook.neo4j_graph_rag:Loaded 3565 nodes and 17719 edges


GraphRAG initialized!
Graph nodes: 3565
Graph edges: 35438


In [12]:
# ============================================================================
# 自然语言查询示例
# ============================================================================

# 示例 1: 查询作业信息
question = "Which file systems did Job 3122490 use? How many data did it read and write?"
result = await rag.query(question, context_depth=2)

print(f"\nAnswer: {result['answer']}")
print(f"\nUsed {len(result['sources']['nodes'])} nodes and {len(result['sources']['edges'])} edges as context")


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Answer: Job 3122490 utilized two distinct file systems based on the data provided in the context: the `/home` file system and the `/lus/grand` file system. This is inferred from the listed files accessed during the job's execution, which include paths from both mount points:

1. Files from `/home`:
   - `/home/3249283385`
   - `/home/3729216590`
   - `/home/2968654182`
   - `/home/925128995`
   - `/home/1191560678`
   - `/home/1795898678`

2. Files from `/lus/grand`:
   - `/lus/grand/3885248573`
   - `/lus/grand/2406118476`
   - `/lus/grand/416614965`
   - `/lus/grand/2697922702`

### Data Read and Written
- **Total Reads**: Job 3122490 performed **530 read operations**, totaling **4,204,192 bytes** read. 
- **Total Writes**: The job did not perform any write operations, resulting in **0 writes** and **0 bytes written**.

### Analysis of I/O Efficiency Indicators
1. **Small I/O Operations**: The job's average operation size is **7,932 bytes**, which is above the problematic threshold 

In [13]:
# Example 2: Query file information
question = "Which files have the largest I/O? What are their characteristics?"
result = await rag.query(question, context_depth=2)

print(f"\nAnswer: {result['answer']}")


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Answer: Based on the provided data from the Darshan I/O profiling logs, we can analyze the files with the largest I/O volumes and their characteristics. 

### Files with the Largest I/O:

1. **File: /home/3139858466**
   - **Total Reads:** 14,147 operations
   - **Total Bytes Read:** 115,864,201 bytes
   - **Total Writes:** 0 operations
   - **Characteristics:** 
     - This file is heavily read but not written to, indicating that it might serve as a data input source rather than a data output target. 
     - The average read size can be determined from the total bytes read divided by the number of read operations: 
       \[
       \text{Avg. Read Size} = \frac{115,864,201 \text{ bytes}}{14,147 \text{ reads}} \approx 8,184 \text{ bytes}
       \]
     - The file is accessed by **Job 3123593** (8 ranks), which may indicate a high demand for this data.

2. **File: /lus/grand/297985663**
   - **Total Reads:** 18 operations
   - **Total Bytes Read:** 6,064 bytes
   - **Total Writes:** 49

In [7]:
import os
import glob
from collections import defaultdict

POSIX_READ = "POSIX_BYTES_READ"
POSIX_WRITE = "POSIX_BYTES_WRITTEN"


def parse_darshan_report(path):
    """
    Parse a single darshan text report.
    Returns:
        total_read_bytes
        total_write_bytes
        file_io_dict: {filename: total_io_bytes}
    """
    total_read = 0
    total_write = 0
    file_io = defaultdict(int)

    with open(path, "r", errors="ignore") as f:
        for line in f:
            if not line.startswith("POSIX"):
                continue

            parts = line.strip().split("\t")
            if len(parts) < 7:
                continue

            _, rank, record_id, counter, value, filename, *_ = parts

            try:
                value = int(value)
            except ValueError:
                continue

            if value < 0:
                continue

            if counter == POSIX_READ:
                total_read += value
                file_io[filename] += value

            elif counter == POSIX_WRITE:
                total_write += value
                file_io[filename] += value

    return total_read, total_write, file_io


def analyze_darshan_reports(report_dir_or_pattern):
    """
    Analyze multiple darshan reports.
    """
    # Original: paths = sorted(glob.glob(report_dir_or_pattern))
    # Fix: Handle case where a directory path is passed without a wildcard
    if os.path.isdir(report_dir_or_pattern):
        # If it's a directory, get all files inside it
        paths = sorted(glob.glob(os.path.join(report_dir_or_pattern, "*")))
        # Filter out subdirectories, keep only files
        paths = [p for p in paths if os.path.isfile(p)]
    else:
        paths = sorted(glob.glob(report_dir_or_pattern))
    
    if not paths:
        raise RuntimeError("No darshan reports found!")

    global_read = 0
    global_write = 0
    global_file_io = defaultdict(int)

    for path in paths:
        r, w, file_io = parse_darshan_report(path)
        global_read += r
        global_write += w

        for fname, io in file_io.items():
            global_file_io[fname] += io

    # file with largest IO
    max_file, max_io = max(global_file_io.items(), key=lambda x: x[1])

    return {
        "total_read_bytes": global_read,
        "total_write_bytes": global_write,
        "max_io_file": max_file,
        "max_io_bytes": max_io,
        # "total_bytes": 
    }


if __name__ == "__main__":
    # Examples:
    # analyze_darshan_reports("reports/*.txt")
    # analyze_darshan_reports("/path/to/darshan/*.log")

    result = analyze_darshan_reports("/users/Minqiu/parsed-logs-2025-1-1")

    print("===== Darshan I/O Summary =====")
    print(f"Total Reads : {result['total_read_bytes']:,} bytes")
    print(f"Total Writes: {result['total_write_bytes']:,} bytes")
    print("Largest I/O File:")
    print(f"  File : {result['max_io_file']}")
    print(f"  I/O  : {result['max_io_bytes']:,} bytes")

===== Darshan I/O Summary =====
Total Reads : 2,785,328,751,890 bytes
Total Writes: 13,919,745,410,209 bytes
Largest I/O File:
  File : /lus/grand/2431195864
  I/O  : 2,783,344,493,776 bytes


In [8]:
import glob
import re
import os

TOTAL_BYTES_RE = re.compile(r"#\s*total_bytes:\s*(\d+)")
# For parsed darshan logs: POSIX_BYTES_READ and POSIX_BYTES_WRITTEN
POSIX_BYTES_READ = "POSIX_BYTES_READ"
POSIX_BYTES_WRITTEN = "POSIX_BYTES_WRITTEN"

def extract_total_bytes(report_path):
    """
    Extract total_bytes from a single darshan text report.
    Supports two formats:
    1. Summary format with '# total_bytes: value'
    2. Parsed darshan format with POSIX_BYTES_READ/WRITTEN entries
    Returns None if not found.
    """
    total_bytes = None
    posix_total = 0
    found_posix = False
    
    with open(report_path, "r", errors="ignore") as f:
        for line in f:
            # Original format: # total_bytes: value
            m = TOTAL_BYTES_RE.search(line)
            if m:
                total_bytes = int(m.group(1))
                return total_bytes  # Return immediately if found
            
            # Parsed darshan format: POSIX\trank\tid\tPOSIX_BYTES_READ/WRITTEN\tvalue\t...
            if line.startswith("POSIX"):
                parts = line.strip().split("\t")
                if len(parts) >= 5:
                    counter = parts[3]
                    if counter in (POSIX_BYTES_READ, POSIX_BYTES_WRITTEN):
                        try:
                            val = int(parts[4])
                            if val >= 0:
                                posix_total += val
                                found_posix = True
                        except ValueError:
                            pass
    
    # Return POSIX total if found, otherwise None
    if found_posix:
        return posix_total
    return None


# Original: def sum_total_bytes(pattern):
def sum_total_bytes(folder_or_pattern):
    """
    Sum total_bytes across multiple darshan reports.
    Also returns the file with maximum total_bytes.
    """
    # Original: paths = sorted(glob.glob(pattern))
    # Fix: Handle folder input
    if os.path.isdir(folder_or_pattern):
        paths = sorted(glob.glob(os.path.join(folder_or_pattern, "*")))
        paths = [p for p in paths if os.path.isfile(p)]
    else:
        paths = sorted(glob.glob(folder_or_pattern))
    
    if not paths:
        raise RuntimeError(f"No reports found for: {folder_or_pattern}")

    per_report = {}
    total = 0

    for p in paths:
        val = extract_total_bytes(p)
        if val is None:
            print(f"[WARN] total_bytes not found in {p}")
            continue

        per_report[p] = val
        total += val

    # Find file with maximum total_bytes
    max_file = None
    max_bytes = 0
    if per_report:
        max_file, max_bytes = max(per_report.items(), key=lambda x: x[1])

    return total, per_report, max_file, max_bytes


if __name__ == "__main__":
    # Example usage - now accepts folder path
    # Original: pattern = "/users/Minqiu/report.txt"
    folder = "/users/Minqiu/parsed-logs-2025-1-1"

    total_bytes, per_report_bytes, max_file, max_bytes = sum_total_bytes(folder)

    print("===== Darshan Total I/O Summary =====")
    for p, v in per_report_bytes.items():
        print(f"{p}: {v:,} bytes")

    print("------------------------------------")
    print(f"GLOBAL TOTAL I/O: {total_bytes:,} bytes")
    print("------------------------------------")
    print(f"MAX I/O FILE: {max_file}")
    print(f"MAX I/O BYTES: {max_bytes:,} bytes")

[WARN] total_bytes not found in /users/Minqiu/parsed-logs-2025-1-1/3123261-6208398433230042948.txt
[WARN] total_bytes not found in /users/Minqiu/parsed-logs-2025-1-1/3123267-12556481513926476372.txt
[WARN] total_bytes not found in /users/Minqiu/parsed-logs-2025-1-1/3123474-3575250355250191983.txt
[WARN] total_bytes not found in /users/Minqiu/parsed-logs-2025-1-1/3123593-5329578837411783626.txt
[WARN] total_bytes not found in /users/Minqiu/parsed-logs-2025-1-1/3123595-3021089735528557529.txt
[WARN] total_bytes not found in /users/Minqiu/parsed-logs-2025-1-1/3123600-10437766810787049059.txt
[WARN] total_bytes not found in /users/Minqiu/parsed-logs-2025-1-1/3123603-2918144303186026878.txt
[WARN] total_bytes not found in /users/Minqiu/parsed-logs-2025-1-1/3123653-10149547465409748855.txt
[WARN] total_bytes not found in /users/Minqiu/parsed-logs-2025-1-1/3123653-10272769247279087403.txt
[WARN] total_bytes not found in /users/Minqiu/parsed-logs-2025-1-1/3123653-10660921444056876305.txt
[WARN

In [5]:
# Example 3:
job_summary = await rag.query_job_summary("3122490")

print("=== Job 3122490 ===")
print(f"\nJob information: {job_summary['job']}")
print(f"\nNumber of files accessed: {len(job_summary['files'])}")
print(f"Number of ranks used: {len(job_summary['ranks'])}")

if job_summary['summary']:
    s = job_summary['summary']
    print(f"\nI/O statistics:")
    print(f"  Total reads: {s.get('total_reads', 0):,}")
    print(f"  Total writes: {s.get('total_writes', 0):,}")
    print(f"  Total bytes read: {s.get('total_bytes_read', 0):,}")
    print(f"  Total bytes written: {s.get('total_bytes_written', 0):,}")

if job_summary['io_pattern']:
    p = job_summary['io_pattern']
    print(f"\nI/O pattern analysis:")
    print(f"  Read/write ratio: {p['read_write_ratio']:.2f}")
    print(f"  Read-heavy ranks: {len(p['read_heavy_ranks'])}")
    print(f"  Write-heavy ranks: {len(p['write_heavy_ranks'])}")


NameError: name 'rag' is not defined

In [ ]:
# Example 4:
file_path = "/lus/grand/1649248601"  # replace with actual file path
file_analysis = await rag.query_file_access_pattern(f"File:{file_path}")

print(f"=== File access analysis: {file_path} ===")
print(f"Jobs accessed this file: {file_analysis['accessed_by_jobs']}")
print(f"Filesystem information: {file_analysis['filesystem']}")

if file_analysis['access_by_rank']:
    print(f"\nRank access statistics:")
    for r in file_analysis['access_by_rank'][:5]:
        stats = r['stats']
        print(f"  {r['rank']}: reads={stats.get('reads', 0)}, writes={stats.get('writes', 0)}")


=== File access analysis: /lus/grand/1649248601 ===
Jobs accessed this file: ['Job:3113659', 'Job:3113659']
Filesystem information: {'mount_point': '/lus/grand', 'filesystem': 'FileSystem:lustre'}


In [16]:
# ============================================================================
# 自定义查询函数 - 你可以修改 question 变量来查询任何问题
# ============================================================================

async def ask_graph(question: str):
    """Use OpenAI to answer questions about the graph"""
    result = await rag.query(question, context_depth=3)
    print(f"\nAnswer:\n{result['answer']}")
    print(f"\n--- Context information ---")
    print(f"Used nodes: {len(result['sources']['nodes'])}")
    print(f"Used edges: {len(result['sources']['edges'])}")
    return result

# 修改这里的问题来进行查询
question = "Which jobs have low I/O efficiency? Why?"
result = await ask_graph(question)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Answer:
To analyze the I/O efficiency of the jobs based on the provided Darshan I/O profiling logs, we should focus on the key indicators of low I/O efficiency: small I/O operations, high seek ratios, imbalanced I/O across ranks, and accessing many files. 

### Job Analysis

1. **Job 3113659**:
   - **Ranks**: 287
   - **Files**: 16
   - **Reads**: 618,672 ops, **Bytes Read**: 2,783,344,552,072
   - **Writes**: 1,660,479 ops, **Bytes Written**: 13,919,337,994,300
   - **Avg Operation Size**: 7,328,467 bytes
   - **Efficiency Assessment**: This job has a very high average operation size, suggesting that it performs large, efficient I/O operations overall. However, the number of ranks is very high relative to the number of files accessed. This could lead to imbalanced I/O across ranks, potentially causing synchronization bottlenecks. 

2. **Jobs 3123595, 3123600, 3123594, 3123601, 3123852** (Grouped together):
   - **Ranks**: 4
   - **Files**: 821
   - **Reads**: 14,161 ops, **Bytes Rea

In [22]:
# ============================================================================
# 【修复】重新加载模块并测试
# ============================================================================

# 强制重新加载模块
import importlib
import sys

# 删除缓存的模块
if 'cookbook.neo4j_graph_rag' in sys.modules:
    del sys.modules['cookbook.neo4j_graph_rag']

# 重新导入
from cookbook.neo4j_graph_rag import (
    create_graph_backend,
    DarshanGraphDB,
    DarshanGraphRAG
)

# 测试实体提取
rag_test = DarshanGraphRAG.__new__(DarshanGraphRAG)
rag_test.backend = None
rag_test.graph_db = None
rag_test.analyzer = None
rag_test.llm_func = None
rag_test.model = "gpt-5.1"

question = "Which jobs have low I/O efficiency? Why?"
entities = rag_test._extract_entities(question)
print(f"entities: {entities}")
print(f"entity count: {len(entities)} (should be 0, if not 0, the regex has problem)")

# 如果实体数量是 0，创建完整的 RAG 并测试
if len(entities) == 0:
    print("\n✅...")
    
    # 创建新后端
    backend_fixed = create_graph_backend('networkx', persist_path='darshan_graph_fixed.json')
    await backend_fixed.connect()
    
    graph_db_fixed = DarshanGraphDB(backend_fixed)
    await graph_db_fixed.load_from_json('graph_2025-1-1.json')
    
    rag_fixed = DarshanGraphRAG(backend_fixed, llm_func=openai_llm, model="gpt-4o-mini")
    
    # 测试 _get_overview_context
    nodes, edges = await rag_fixed._get_overview_context()
    print(f"\n_get_overview_context results:")
    print(f"  nodes: {len(nodes)}")
    print(f"  edges: {len(edges)}")
    
    # 显示节点类型
    from collections import Counter
    node_types = Counter(n.node_type for n in nodes)
    print(f"  node types: {dict(node_types)}")
    
    print("\n✅...")
else:
    print(f"\n❌ Error, still have problem with entity extraction: {entities}")

INFO:cookbook.neo4j_graph_rag:NetworkX backend initialized


INFO:cookbook.neo4j_graph_rag:Loaded 3565 nodes and 17719 edges


entities: []
entity count: 0 (should be 0, if not 0, the regex has problem)

✅...

_get_overview_context results:
  nodes: 447
  edges: 356
  node types: {'JobIOSummary': 67, 'Job': 67, 'File': 10, 'FileSystem': 16, 'Rank': 287}

✅...


In [23]:
# ============================================================================
# 测试修复后的通用问题查询
# ============================================================================

# 现在可以问通用问题了！
question = "Which jobs have low I/O efficiency? Why?"
result = await rag_new.query(question, context_depth=2)

print(f"\nAnswer:\n{result['answer']}")
print(f"\n--- Context information ---")
print(f"Used nodes: {len(result['sources']['nodes'])}")
print(f"Used edges: {len(result['sources']['edges'])}")


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Answer:
Based on the provided I/O profiling logs and relevant metrics, we can identify several jobs that exhibit low I/O efficiency due to various inefficiencies in their I/O patterns. Here are the key findings for specific jobs:

### Job 3123595
- **Ranks:** 4
- **Files:** 821
- **Reads:** 14,161 operations, 115,890,934 bytes
- **Writes:** 34 operations, 5,205 bytes
- **Avg Operation Size:** 8,165 bytes

**Issues Identified:**
- **Imbalanced I/O:** This job has a large number of files accessed (821) with very few write operations (only 34 writes). This indicates potential metadata overhead due to managing a large number of files, which can be inefficient if the files are not utilized effectively.
- **Small number of operations:** The number of read operations (14,161) compared to the very low number of write operations (34) suggests that the job is read-heavy but not writing back significant data, which may indicate poor data handling practices or a design that does not utilize the w

In [24]:
# ============================================================================
# 更多通用问题示例（使用修复后的 rag_new）
# ============================================================================

async def ask_new(question: str):
    result = await rag_new.query(question, context_depth=2)
    print(f"\nAnswer:\n{result['answer']}")
    print(f"\n[Context: {len(result['sources']['nodes'])} nodes, {len(result['sources']['edges'])} edges]")
    print("-" * 70)
    return result

# 示例通用问题
questions = [
    "Which job has the largest I/O? What are its characteristics?",
    "Are there any jobs with small I/O problems?",
    "What file systems did all jobs use?",
]

# 运行第一个问题（可以修改索引测试其他问题）
await ask_new(questions[0])


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Answer:
The job with the largest I/O is **Job 3113659**. Here are its characteristics based on the provided I/O profiling logs:

### Job Characteristics:
- **Job ID:** 3113659
- **Number of Ranks:** 287
- **Number of Files:** 16
- **Total Read Operations:** 618,672 operations
- **Total Write Operations:** 1,660,479 operations
- **Bytes Read:** 2,783,344,552,072 bytes (approximately 2.78 TB)
- **Bytes Written:** 13,919,337,994,300 bytes (approximately 13.92 TB)
- **Average Operation Size:** 7,328,467 bytes (approximately 7.33 MB)

### Analysis:
1. **I/O Volume:**
   - Job 3113659 exhibits an extremely high volume of I/O with a combined total of approximately 16.7 TB (2.78 TB read + 13.92 TB written). This is significantly higher than any other job in the context provided.

2. **Average Operation Size:**
   - The average I/O operation size is 7.33 MB, which is well above the threshold of 4 KB. This indicates that the job is performing relatively efficient I/O operations, as larger opera

{'answer': 'The job with the largest I/O is **Job 3113659**. Here are its characteristics based on the provided I/O profiling logs:\n\n### Job Characteristics:\n- **Job ID:** 3113659\n- **Number of Ranks:** 287\n- **Number of Files:** 16\n- **Total Read Operations:** 618,672 operations\n- **Total Write Operations:** 1,660,479 operations\n- **Bytes Read:** 2,783,344,552,072 bytes (approximately 2.78 TB)\n- **Bytes Written:** 13,919,337,994,300 bytes (approximately 13.92 TB)\n- **Average Operation Size:** 7,328,467 bytes (approximately 7.33 MB)\n\n### Analysis:\n1. **I/O Volume:**\n   - Job 3113659 exhibits an extremely high volume of I/O with a combined total of approximately 16.7 TB (2.78 TB read + 13.92 TB written). This is significantly higher than any other job in the context provided.\n\n2. **Average Operation Size:**\n   - The average I/O operation size is 7.33 MB, which is well above the threshold of 4 KB. This indicates that the job is performing relatively efficient I/O operati

In [25]:
# ============================================================================
# 测试修复后的通用问题查询
# ============================================================================

# 现在可以问通用问题了！
question = "Group jobs into clusters of jobs that were executed by the same user and have roughly similar characteristics (approximate scale, execution time, number of files, volume of data read and written).  Assess the I/O performance variability within each cluster.  What is the distribution of I/O variability across clusters?  What are the high-level I/O characteristics of clusters with higher variability than others?"
result = await rag_new.query(question, context_depth=2)

print(f"\nAnswer:\n{result['answer']}")
print(f"\n--- Context information ---")
print(f"Used nodes: {len(result['sources']['nodes'])}")
print(f"Used edges: {len(result['sources']['edges'])}")


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Answer:
To analyze the I/O performance of the jobs based on the provided data, we will group the jobs into clusters based on their characteristics, such as the number of ranks, number of files accessed, volume of data read and written, and average operation size. We will assess the I/O performance variability within each cluster and discuss the distribution of I/O variability across clusters.

### Clustering Jobs

#### Cluster 1: High Volume I/O Jobs
- **Job 3113659**
  - Ranks: 287
  - Files: 16
  - Reads: 618,672 ops, 2,783,344,552,072 bytes
  - Writes: 1,660,479 ops, 13,919,337,994,300 bytes
  - Avg operation size: 7,328,467 bytes

This job stands out due to its high aggregate I/O volume, both in terms of reads and writes. It has a small number of files (16) but a very high number of operations. The average operation size is also significantly high, indicating that the access patterns are likely efficient for this job.

#### Cluster 2: Low Volume I/O Jobs
- **Jobs 3123595, 3123600,